In [136]:
%load_ext autoreload
%autoreload 2
import theano
import theano.tensor as T
import numpy as np
import SRCNN_Theano
import os
import preprocessing
from scipy import ndimage,misc


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#Create Validation_Subsamples_RGB_4 and Validation_Subsamples_RGB_4_GT folders
preprocessing.create_subimages('/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Set5',
'/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Validation_Subsamples_RGB_4',
output_type ='RGB',upsampling_factor = 4)
preprocessing.create_subimages('/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Set14',
'/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Test_Subsamples_RGB_4',
output_type ='RGB',upsampling_factor = 4)
preprocessing.create_subimages('/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Training_Full',
'/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Training_Subsamples_RGB_4',
output_type ='RGB',upsampling_factor = 4)

/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Set5
processed:0
/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Set14
processed:0
processed:10
/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Training_Full
processed:0
processed:10
processed:20
processed:30
processed:40
processed:50
processed:60
processed:70
processed:80
processed:90


In [3]:
import SRCNN_Theano

#load dataset

#load training
data_x = SRCNN_Theano.load_dataset('Data/Training_Subsamples_RGB_4','data_x')
data_y = SRCNN_Theano.load_dataset('Data/Training_Subsamples_RGB_4_gt','data_y')
valid_x = SRCNN_Theano.load_dataset('Data/Validation_Subsamples_RGB_4','data_x')
valid_y = SRCNN_Theano.load_dataset('Data/Validation_Subsamples_RGB_4_gt','data_y')
test_x = SRCNN_Theano.load_dataset('Data/Test_Subsamples_RGB_4','data_x')
test_y = SRCNN_Theano.load_dataset('Data/Test_Subsamples_RGB_4_gt','data_y')
print "done loading\n\n"
print "data_x: " + str(data_x.shape)
print "data_y: " + str(data_y.shape)
print "valid_x: " + str(valid_x.shape)
print "valid_y: " + str(valid_y.shape)
print "test_x: " + str(test_x.shape)
print "test_y: " + str(test_y.shape)

/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Training_Subsamples_RGB_4
loading from npz
/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Training_Subsamples_RGB_4_gt
loading from npz
/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Validation_Subsamples_RGB_4
loading from npz
/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Validation_Subsamples_RGB_4_gt
loading from npz
/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Test_Subsamples_RGB_4
loading from npz
/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Test_Subsamples_RGB_4_gt
loading from npz
done loading


data_x: (22092, 3, 8, 8)
data_y: (22092, 3, 33, 33)
valid_x: (2488, 3, 8, 8)
valid_y: (2488, 3, 33, 33)
test_x: (14851, 3, 8, 8)
test_y: (14851, 3, 33, 33)


In [17]:
#Bicubic interp to save computation during training
upsampled_x = preprocessing.upsample(data_x) #33,33,3 input images expected
up_val_x = preprocessing.upsample(valid_x) #33,33,3 input images expected
up_test_x = preprocessing.upsample(test_x) #33,33,3 input images expected

#Reshape for training,valid,test
upsampled_x = upsampled_x.reshape((22092,33*33*3))
data_y = data_y.reshape((22092,33*33*3))
up_val_x = up_val_x.reshape((2488,33*33*3))
valid_y = valid_y.reshape((2488,33*33*3))
up_test_x = up_test_x.reshape((14851,33*33*3))
test_y = test_y.reshape((14851,33*33*3))

print upsampled_x.shape
print data_y.shape
print up_val_x.shape
print up_test_x.shape

(22092, 3267)
(22092, 3267)
(2488, 3267)
(14851, 3267)


In [5]:
shared_x = theano.shared(np.asarray(upsampled_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_y = theano.shared(np.asarray(data_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_x = theano.shared(np.asarray(up_val_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_val_y = theano.shared(np.asarray(valid_y,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_x = theano.shared(np.asarray(up_test_x,
                                       dtype=theano.config.floatX),
                         borrow=True)
shared_test_y = theano.shared(np.asarray(test_y,
                                       dtype=theano.config.floatX),
                         borrow=True)

# Batch Normalization with L2 parameter search and RMSProp

In [16]:
import SRCNN_Theano
batch_size = 20
n_epochs = 10

lr = 3e-5
l2s = [0,.1,1e-2,1e-3,1e-4]
for l2 in l2s:
    print "\n\n ****************************** l2 = " + str(l2) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    train_model,val_model,test_model = SRCNN_Theano.train_BN_SRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,l2,upsampling_factor=4)




 ****************************** l2 = 0******************************************
theano optimizer: fast_compile
epoch 1, minibatch 1104/1104, validation cost 94.890968 mse/pixel: 0.071724 pnsr: 11.928905
     epoch 1, minibatch 1104/1104, test cost of best model 115.416977 perpixel mse 0.087239 and test pnsr 11.812642
epoch 2, minibatch 1104/1104, validation cost 83.633438 mse/pixel: 0.063215 pnsr: 12.468738
     epoch 2, minibatch 1104/1104, test cost of best model 94.270187 perpixel mse 0.071255 and test pnsr 12.675542
epoch 3, minibatch 1104/1104, validation cost 79.539391 mse/pixel: 0.060120 pnsr: 12.733826
     epoch 3, minibatch 1104/1104, test cost of best model 87.609428 perpixel mse 0.066220 and test pnsr 12.994287
epoch 4, minibatch 1104/1104, validation cost 77.405128 mse/pixel: 0.058507 pnsr: 12.863464
     epoch 4, minibatch 1104/1104, test cost of best model 85.133705 perpixel mse 0.064349 and test pnsr 13.089473
epoch 5, minibatch 1104/1104, validation cost 74.266769 m

# Batch Normalization learning rate search

In [20]:
batch_size = 20
n_epochs = 10

lrs = [1e-2,1e-3,1e-4,3e-5]
l2 = 0
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    train_model,val_model,test_model = SRCNN_Theano.train_BN_SRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,l2,upsampling_factor=4)




 ****************************** lr = 0.01******************************************
theano optimizer: fast_compile
epoch 1, minibatch 1104/1104, validation cost 80.526855 mse/pixel: 0.060867 pnsr: 12.713138
     epoch 1, minibatch 1104/1104, test cost of best model 87.853172 perpixel mse 0.066405 and test pnsr 13.086980
epoch 2, minibatch 1104/1104, validation cost 75.473412 mse/pixel: 0.057047 pnsr: 13.023520
     epoch 2, minibatch 1104/1104, test cost of best model 83.438988 perpixel mse 0.063068 and test pnsr 13.337627
epoch 3, minibatch 1104/1104, validation cost 72.225739 mse/pixel: 0.054592 pnsr: 13.209533
     epoch 3, minibatch 1104/1104, test cost of best model 81.032234 perpixel mse 0.061249 and test pnsr 13.437563
epoch 4, minibatch 1104/1104, validation cost 70.284454 mse/pixel: 0.053125 pnsr: 13.342794
     epoch 4, minibatch 1104/1104, test cost of best model 79.108650 perpixel mse 0.059795 and test pnsr 13.538618
epoch 5, minibatch 1104/1104, validation cost 66.858162

# RMS optimizer with l2 parameter search

In [21]:
import SRCNN_Theano
batch_size = 20
n_epochs = 10

lr = 3e-5
l2s = [0,.1,1e-2,1e-3,1e-4]
for l2 in l2s:
    print "\n\n ****************************** l2 = " + str(l2) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    train_model,val_model,test_model = SRCNN_Theano.train_SRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,l2,upsampling_factor=4)




 ****************************** l2 = 0******************************************
theano optimizer: fast_compile
epoch 1, minibatch 1104/1104, validation cost 85.796295 mse/pixel: 85.796295 pnsr: 29.655224
     epoch 1, minibatch 1104/1104, test cost of best model 107.629440 perpixel mse 107.629440 and test pnsr 28.861370
epoch 2, minibatch 1104/1104, validation cost 48.743275 mse/pixel: 48.743275 pnsr: 32.543144
     epoch 2, minibatch 1104/1104, test cost of best model 60.354828 perpixel mse 60.354828 and test pnsr 31.828428
epoch 3, minibatch 1104/1104, validation cost 45.359776 mse/pixel: 45.359776 pnsr: 32.668751
     epoch 3, minibatch 1104/1104, test cost of best model 55.568443 perpixel mse 55.568443 and test pnsr 32.035465
epoch 4, minibatch 1104/1104, validation cost 43.751617 mse/pixel: 43.751617 pnsr: 32.771458
     epoch 4, minibatch 1104/1104, test cost of best model 53.942047 perpixel mse 53.942047 and test pnsr 32.095943
epoch 5, minibatch 1104/1104, validation cost 46

# Adam Learning rate search with batch size 20

In [23]:
batch_size = 20
n_epochs = 10
lrs = [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    train_model,val_model,test_model = SRCNN_Theano.train_SRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)



 ****************************** lr = 0.1******************************************
theano optimizer: fast_compile
epoch 1, minibatch 1104/1104, validation cost 5818.668945 mse/pixel: 5818.668945 pnsr: 10.977448
     epoch 1, minibatch 1104/1104, test cost of best model 4792.900879 perpixel mse 4792.900879 and test pnsr 12.603998
epoch 2, minibatch 1104/1104, validation cost 5661.235352 mse/pixel: 5661.235352 pnsr: 11.037873
     epoch 2, minibatch 1104/1104, test cost of best model 4610.493652 perpixel mse 4610.493652 and test pnsr 12.724049
epoch 3, minibatch 1104/1104, validation cost 5508.677734 mse/pixel: 5508.677734 pnsr: 11.094985
     epoch 3, minibatch 1104/1104, test cost of best model 4428.395020 perpixel mse 4428.395020 and test pnsr 12.831921
epoch 4, minibatch 1104/1104, validation cost 5444.961914 mse/pixel: 5444.961914 pnsr: 11.120966
     epoch 4, minibatch 1104/1104, test cost of best model 4352.605957 perpixel mse 4352.605957 and test pnsr 12.873102
epoch 5, minibat

In [29]:
batch_size = 20
n_epochs = 50
lrs = [1e-4]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    train_model,val_model,test_model = SRCNN_Theano.train_SRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)



 ****************************** lr = 0.0001******************************************
theano optimizer: fast_compile
epoch 1, minibatch 1104/1104, validation cost 149.853867 mse/pixel: 149.853867 pnsr: 27.013834
     epoch 1, minibatch 1104/1104, test cost of best model 199.465469 perpixel mse 199.465469 and test pnsr 25.775974
epoch 2, minibatch 1104/1104, validation cost 40.256367 mse/pixel: 40.256367 pnsr: 33.085915
     epoch 2, minibatch 1104/1104, test cost of best model 53.115585 perpixel mse 53.115585 and test pnsr 32.141129
epoch 3, minibatch 1104/1104, validation cost 36.682652 mse/pixel: 36.682652 pnsr: 33.719696
     epoch 3, minibatch 1104/1104, test cost of best model 47.851200 perpixel mse 47.851200 and test pnsr 32.747974
epoch 4, minibatch 1104/1104, validation cost 37.245152 mse/pixel: 37.245152 pnsr: 33.500301
epoch 5, minibatch 1104/1104, validation cost 34.539547 mse/pixel: 34.539547 pnsr: 33.948322
     epoch 5, minibatch 1104/1104, test cost of best model 46.53

In [8]:
###rms with xavier

batch_size = 20
n_epochs = 10
lrs = [.1,.07,.05,.03,.007,.005,.002,.0009,.0007,.0004,.0001]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    train_model,val_model,test_model = SRCNN_Theano.train_SRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)



 ****************************** lr = 0.1******************************************
theano optimizer: fast_compile
epoch 1, minibatch 1104/1104, validation cost 5219.087402 mse/pixel: 5219.087402 pnsr: 11.388492
     epoch 1, minibatch 1104/1104, test cost of best model 4603.319824 perpixel mse 4603.319824 and test pnsr 12.710751
epoch 2, minibatch 1104/1104, validation cost 5151.449707 mse/pixel: 5151.449707 pnsr: 11.409640
     epoch 2, minibatch 1104/1104, test cost of best model 4527.587402 perpixel mse 4527.587402 and test pnsr 12.727822
epoch 3, minibatch 1104/1104, validation cost 5140.902832 mse/pixel: 5140.902832 pnsr: 11.414274
     epoch 3, minibatch 1104/1104, test cost of best model 4513.727051 perpixel mse 4513.727051 and test pnsr 12.733249
epoch 4, minibatch 1104/1104, validation cost 5138.301270 mse/pixel: 5138.301270 pnsr: 11.415654
     epoch 4, minibatch 1104/1104, test cost of best model 4510.058594 perpixel mse 4510.058594 and test pnsr 12.735238
epoch 5, minibat

In [9]:
#RMS with xavier

batch_size = 20
n_epochs = 10
lrs = [1e-4,8e-5,5e-5,3e-5,1e-5,1e-6,1e-7]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size



    train_model,val_model,test_model = SRCNN_Theano.train_SRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)



 ****************************** lr = 0.0001******************************************
theano optimizer: fast_compile
epoch 1, minibatch 1104/1104, validation cost 103.781960 mse/pixel: 103.781960 pnsr: 29.011488
     epoch 1, minibatch 1104/1104, test cost of best model 123.437088 perpixel mse 123.437088 and test pnsr 28.514578
epoch 2, minibatch 1104/1104, validation cost 85.370499 mse/pixel: 85.370499 pnsr: 29.592093
     epoch 2, minibatch 1104/1104, test cost of best model 111.226677 perpixel mse 111.226677 and test pnsr 28.728579
epoch 3, minibatch 1104/1104, validation cost 76.613930 mse/pixel: 76.613930 pnsr: 30.068529
     epoch 3, minibatch 1104/1104, test cost of best model 96.425827 perpixel mse 96.425827 and test pnsr 29.302635
epoch 4, minibatch 1104/1104, validation cost 90.955788 mse/pixel: 90.955788 pnsr: 29.270496
epoch 5, minibatch 1104/1104, validation cost 85.883804 mse/pixel: 85.883804 pnsr: 29.579493
epoch 6, minibatch 1104/1104, validation cost 90.033218 mse/pi

In [10]:
#RMS with xavier

batch_size = 20
n_epochs = 100
lrs = [3e-5]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size

    train_model,val_model,test_model = SRCNN_Theano.train_SRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)



 ****************************** lr = 3e-05******************************************
theano optimizer: fast_compile
epoch 1, minibatch 1104/1104, validation cost 100.131859 mse/pixel: 100.131859 pnsr: 28.855673
     epoch 1, minibatch 1104/1104, test cost of best model 129.126129 perpixel mse 129.126129 and test pnsr 27.920048
epoch 2, minibatch 1104/1104, validation cost 53.825432 mse/pixel: 53.825432 pnsr: 32.094799
     epoch 2, minibatch 1104/1104, test cost of best model 66.722565 perpixel mse 66.722565 and test pnsr 31.430304
epoch 3, minibatch 1104/1104, validation cost 49.635342 mse/pixel: 49.635342 pnsr: 32.266251
     epoch 3, minibatch 1104/1104, test cost of best model 61.129517 perpixel mse 61.129517 and test pnsr 31.688190
epoch 4, minibatch 1104/1104, validation cost 48.425186 mse/pixel: 48.425186 pnsr: 32.281681
     epoch 4, minibatch 1104/1104, test cost of best model 58.959206 perpixel mse 58.959206 and test pnsr 31.769852
epoch 5, minibatch 1104/1104, validation c

In [22]:
#RMS with xavier
batch_size = 20
n_epochs = 100
lrs = [3e-5]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size

    train_model,val_model,test_model = SRCNN_Theano.train_SRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,1e-4,upsampling_factor=4)



 ****************************** lr = 3e-05******************************************
theano optimizer: fast_compile
epoch 1, minibatch 1104/1104, validation cost 95.799446 mse/pixel: 95.798958 pnsr: 29.077272
     epoch 1, minibatch 1104/1104, test cost of best model 125.571220 perpixel mse 125.570732 and test pnsr 28.023024
epoch 2, minibatch 1104/1104, validation cost 48.495106 mse/pixel: 48.494625 pnsr: 32.553944
     epoch 2, minibatch 1104/1104, test cost of best model 60.516342 perpixel mse 60.515858 and test pnsr 31.765141
epoch 3, minibatch 1104/1104, validation cost 44.868523 mse/pixel: 44.868038 pnsr: 32.705563
     epoch 3, minibatch 1104/1104, test cost of best model 55.982559 perpixel mse 55.982079 and test pnsr 31.957731
epoch 4, minibatch 1104/1104, validation cost 43.497330 mse/pixel: 43.496841 pnsr: 32.792660
     epoch 4, minibatch 1104/1104, test cost of best model 54.127300 perpixel mse 54.126812 and test pnsr 32.070415
epoch 5, minibatch 1104/1104, validation cos

In [11]:
reconstructed_imgs = np.zeros((14851, 3, 21, 21))
for i in xrange(n_test_batches):
     cost,MSE_per_pixel,psnr,reconstucted_patches = test_model(i)
     reconstructed_imgs[i*batch_size:(i+1)*batch_size,:,:,:] = reconstucted_patches

SRCNN_Theano.rebuild_images(reconstructed_imgs,'/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Test_Subsamples_RGB_4',patch_dim=21,dataset='test')
        

In [15]:
up_test_x = preprocessing.upsample(test_x) #33,33,3 input images expected
SRCNN_Theano.rebuild_images(up_test_x,'/home/jon/Documents/Neural_Networks/project/SRCNN_Theano/Data/Test_Subsamples_RGB_4',patch_dim=33,dataset='test')


In [30]:
#SGD with xavier

batch_size = 20
n_epochs = 50
lrs = [1e-4]
for lr in lrs:
    print "\n\n ****************************** lr = " + str(lr) +"******************************************"
    learning_rate = lr

    n_train_batches = upsampled_x.shape[0]/batch_size
    n_valid_batches = up_val_x.shape[0]/batch_size
    n_test_batches = up_test_x.shape[0]/batch_size

    train_model,val_model,test_model = SRCNN_Theano.train_SRCNN(shared_x,shared_y,
                             shared_val_x,shared_val_y,
                             shared_test_x,shared_test_y,
                            n_train_batches, n_valid_batches, n_test_batches, 
                             n_epochs, batch_size,learning_rate,upsampling_factor=4)



 ****************************** lr = 0.0001******************************************
theano optimizer: fast_compile
epoch 1, minibatch 1104/1104, validation cost 17780.742188 mse/pixel: 17780.742188 pnsr: 6.950961
     epoch 1, minibatch 1104/1104, test cost of best model 17394.910156 perpixel mse 17394.910156 and test pnsr 6.478099
epoch 2, minibatch 1104/1104, validation cost 17778.291016 mse/pixel: 17778.291016 pnsr: 6.951791
     epoch 2, minibatch 1104/1104, test cost of best model 17392.380859 perpixel mse 17392.380859 and test pnsr 6.478842
epoch 3, minibatch 1104/1104, validation cost 17775.835938 mse/pixel: 17775.835938 pnsr: 6.952621
     epoch 3, minibatch 1104/1104, test cost of best model 17389.851562 perpixel mse 17389.851562 and test pnsr 6.479583
epoch 4, minibatch 1104/1104, validation cost 17773.388672 mse/pixel: 17773.388672 pnsr: 6.953451
     epoch 4, minibatch 1104/1104, test cost of best model 17387.324219 perpixel mse 17387.324219 and test pnsr 6.480325
epoch